# Iterative Prompt Development
When we are writing prompts to develop an application using LLM, the process can be quite similar where we have an idea for what we want to do, the task we want to complete, and we can then take a first attempt at writing a prompt that hopefully is clear and specific, and maybe, if appropriate, gives the system time to think. And then we can run it and see what result we get. And if it doesn't work well enough the first time, then the iterative process of figuring out why the instructions, for example, were not clear enough or why it didn’t give the algorithm enough time to think, allows us to refine the idea, refine the prompt and so on, and to go around this loop multiple times until we end up with a prompt that works for our application.

In [12]:
import openai
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.getenv('OPENAI_API_KEY')

In [13]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

## Generate a marketing product description from a product fact sheet

In [14]:
fact_sheet_chair = """
OVERVIEW
- Part of a beautiful family of mid-century inspired office furniture, 
including filing cabinets, desks, bookcases, meeting tables, and more.
- Several options of shell color and base finishes.
- Available with plastic back and front upholstery (SWC-100) 
or full upholstery (SWC-110) in 10 fabric and 6 leather options.
- Base finish options are: stainless steel, matte black, 
gloss white, or chrome.
- Chair is available with or without armrests.
- Suitable for home or business settings.
- Qualified for contract use.

CONSTRUCTION
- 5-wheel plastic coated aluminum base.
- Pneumatic chair adjust for easy raise/lower action.

DIMENSIONS
- WIDTH 53 CM | 20.87”
- DEPTH 51 CM | 20.08”
- HEIGHT 80 CM | 31.50”
- SEAT HEIGHT 44 CM | 17.32”
- SEAT DEPTH 41 CM | 16.14”

OPTIONS
- Soft or hard-floor caster options.
- Two choices of seat foam densities: 
 medium (1.8 lb/ft3) or high (2.8 lb/ft3)
- Armless or 8 position PU armrests 

MATERIALS
SHELL BASE GLIDER
- Cast Aluminum with modified nylon PA6/PA66 coating.
- Shell thickness: 10 mm.
SEAT
- HD36 foam

COUNTRY OF ORIGIN
- Italy
"""

In [15]:
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

Technical specifications: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)


Introducing our stylish and versatile mid-century inspired office chair, perfect for both home and business settings. This chair is part of a beautiful family of office furniture that includes filing cabinets, desks, bookcases, meeting tables, and more.

Customize your chair with several options of shell color and base finishes to suit your personal style. Choose between plastic back and front upholstery or full upholstery in a variety of fabric and leather options. The base finish options include stainless steel, matte black, gloss white, or chrome. You can also choose to have armrests or go for a sleek armless design.

Constructed with a 5-wheel plastic coated aluminum base and featuring a pneumatic chair adjust for easy raise/lower action, this chair is both durable and functional. The dimensions of the chair are as follows: WIDTH 53 CM | 20.87”, DEPTH 51 CM | 20.08”, HEIGHT 80 CM | 31.50”, SEAT HEIGHT 44 CM | 17.32”, SEAT DEPTH 41 CM | 16.14”.

Customize your chair further with opt

## Issue 1: The text is too long 
This was our first attempt in writing a prompt to the model but the response in quite long. Maybe we want it to be a little bit shorter. SO here we will clarify the prompt by adding a line, `"Use atmost 50 words."`.

In [16]:
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

Use at most 50 words.

Technical specifications: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)


Introducing our versatile and stylish office chair, part of a mid-century inspired furniture collection. Available in various colors and finishes, with options for upholstery and armrests. Designed for both home and business use, with a durable construction and adjustable height feature. Made in Italy for quality assurance.


This actually looks much nice and short description. Let's check the length of the response.

In [17]:
len(response.split())

47

## Issue 2. Text focuses on the wrong details
LLMs are not that great at following instructions about precise word count. But this is actually not bad. Sometimes it will print out something with 60 or 65 so on words, but it's kind of within reason. Some of the things we could try to do would be to say "use atmost three sentences" or "use atmost 280 characters".
Let's take the prompt and change it such that we want more precise abouth the technical details about the furniture.

In [18]:
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

The description is intended for furniture retailers, 
so should be technical in nature and focus on the 
materials the product is constructed from.

Use at most 50 words.

Technical specifications: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)

Introducing our versatile and stylish office chair, part of a mid-century inspired furniture collection. Choose from a variety of shell colors and base finishes to suit your space. Constructed with durable materials like cast aluminum and HD36 foam, this chair is perfect for both home and business settings. Made in Italy.


We can further imporve the prompt by adding the product IDs.

In [19]:
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

The description is intended for furniture retailers, 
so should be technical in nature and focus on the 
materials the product is constructed from.

At the end of the description, include every 7-character 
Product ID in the technical specification.

Use at most 50 words.

Technical specifications: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)

Introducing our versatile and stylish office chair, featuring a durable cast aluminum shell with a nylon coating and comfortable HD36 foam seat. Choose from a variety of base finishes and upholstery options to suit your space. Perfect for home or business use. Product IDs: SWC-100, SWC-110.


## Issue 3. Description needs a table of dimensions
We can also ask it to extract information and organize it in a table.

In [20]:
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

The description is intended for furniture retailers, 
so should be technical in nature and focus on the 
materials the product is constructed from.

At the end of the description, include every 7-character 
Product ID in the technical specification.

After the description, include a table that gives the 
product's dimensions. The table should have two columns.
In the first column include the name of the dimension. 
In the second column include the measurements in inches only.

Give the table the title 'Product Dimensions'.

Format everything as HTML that can be used in a website. 
Place the description in a <div> element.

Technical specifications: ```{fact_sheet_chair}```
"""

response = get_completion(prompt)
print(response)

<div>
<p>Introducing our latest addition to the mid-century inspired office furniture collection - the versatile and stylish office chair. This chair is part of a family of furniture pieces that includes filing cabinets, desks, bookcases, meeting tables, and more, all designed with a mid-century aesthetic in mind.</p>

<p>Available in a variety of shell colors and base finishes, this chair can be customized to suit any space. Choose between plastic back and front upholstery or full upholstery in a range of fabric and leather options. The base finish options include stainless steel, matte black, gloss white, or chrome, allowing you to match the chair to your existing decor.</p>

<p>Constructed with a 5-wheel plastic coated aluminum base and featuring a pneumatic chair adjust for easy raise/lower action, this chair is not only stylish but also functional. Whether you need a chair for your home office or a business setting, this chair is suitable for both. Plus, it is qualified for contra

## Load Python libraries to view HTML

In [21]:
from IPython.display import display, HTML

In [22]:
display(HTML(response))

Dimension,Measurements (inches)
WIDTH,"20.87"""
DEPTH,"20.08"""
HEIGHT,"31.50"""
SEAT HEIGHT,"17.32"""
SEAT DEPTH,"16.14"""
